# Using Binance API feeding

Here goes the link, explanation and all that.
https://github.com/sammchardy/python-binance

In [1]:
# here goes all the imports just because
import json
from binance.client import Client
from externalized_functions import get_historical_klines, date_to_milliseconds, get_futures_historical_klines
import pandas as pd


print('feed me daddy')

feed me daddy


## Using a Client object to get historical data
It´s not necessary to create an API to get historical data from Binance, we can use a dummy client with
empty '' keys to get data.

In [4]:
# set some parameters
symbol = "BTCUSDT"  # trade-pair to extract
start = "9 Sep, 2019"  # yes
end = "1 Sep, 2020"  # uhu
interval = Client.KLINE_INTERVAL_1MINUTE  # granularity
fileout = "Binance_{}_{}_{}-{}.csv".format(symbol, interval, date_to_milliseconds(start), date_to_milliseconds(end))

# let's get it
lines = get_futures_historical_klines(symbol, interval, start, end)

# convert to dataframe so we can keep playing and save it to a csv file
columns = ['Open_time', 'Open', 'High', 'Low','Close', 'Volume', 'Close_time', 'Quote_asset_volume',
           'Number_of_trades', 'Taker_buy_base_asset_volume', 'Taker_buy_quote_asset_volume', 'Ignore']
df = pd.DataFrame(lines, columns=columns)
df.to_csv(fileout, index=False)

KeyboardInterrupt: 

## Plotting some data extracted

Let´s make some graphs just out of curiosity.

In [2]:
import finplot as fplt
import numpy as np
import pandas as pd

symbol = 'FUTURES BTC-USDT'

#df = pd.read_csv('Binance_BTCUSDT_1m_1596240000000-1598918400000.csv')
df = pd.read_csv('Binance_BTCUSDT_1m_1567987200000-1598918400000.csv', nrows=5000)
df = df.rename(columns={'Open_time':'time', 'Open':'open', 'Close':'close', 'High':'high', 'Low':'low', 'Volume':'volume'})
#df = df.astype({'time':'datetime64[ns]'})
df
#df
#data
#print('here goes the graph')

,time,open,high,low,close,volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,Ignore
0,1567987200000,10391.63,10391.63,10391.63,10391.63,0.000,1567987259999,0.00000,0,0.000,0.00000,0
1,1567987260000,10391.63,10391.63,10391.63,10391.63,0.000,1567987319999,0.00000,0,0.000,0.00000,0
2,1567987320000,10391.63,10391.63,10391.63,10391.63,0.000,1567987379999,0.00000,0,0.000,0.00000,0
3,1567987380000,10391.63,10391.63,10391.63,10391.63,0.000,1567987439999,0.00000,0,0.000,0.00000,0
4,1567987440000,10391.63,10391.63,10391.63,10391.63,0.000,1567987499999,0.00000,0,0.000,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1568286900000,10142.11,10148.81,10140.85,10147.68,12.547,1568286959999,127283.88392,13,11.030,111895.71123,0
4996,1568286960000,10145.58,10148.82,10143.83,10144.60,8.331,1568287019999,84533.62714,13,4.908,49805.97702,0
4997,1568287020000,10145.16,10148.79,10143.95,10148.79,9.375,1568287079999,95122.95606,15,8.383,85059.02888,0
4998,1568287080000,10146.56,10148.93,10146.52,10148.19,11.300,1568287139999,114671.86019,19,10.657,108147.63911,0


In [3]:
# calculate indicators

windows = [3, 5, 10, 30, 60, 60*4, 60*12, 60*24, 60*24*3, 60*24*7, 60*24*30]

maxim = []
minim = []
center = []
volatility_diff = []
volatility = []

for window in windows: 
    ma = df['high'].rolling(window).max() # volatility box top
    mi = df['low'].rolling(window).min()  # volatility box bottom}
    c  = ma-mi # volatility box center
    vol = c/ma # volatility
    diff = vol.diff()
    
    volatility.append(vol)
    volatility_diff.append(diff)
    #volatility = volatility.rolling(60*6).mean()
    #open_price_position_relative_to_volatility_box = (df['open']-df['low'])/center

In [5]:
# create two plots
ax, ax2, ax3 = fplt.create_plot(symbol, rows=3)

# plot candle sticks
candles = df[['time','open','close','high','low']]
fplt.candlestick_ochl(candles, ax=ax)

# overlay volume on the top plot
volumes = df[['time','open','close','volume']]
fplt.volume_ocv(volumes, ax=ax.overlay())

# put an MA on the close price
#fplt.plot(df['time'], df['close'].rolling(window).mean(), ax=ax, legend='ma-25')
#fplt.plot(df['time'], maxim, ax=ax, legend='maxim')
#fplt.plot(df['time'], minim, ax=ax, legend='minim')

fplt.plot(df['time'], stop_high, ax=ax, legend='stop high')
fplt.plot(df['time'], stop_low, ax=ax, legend='stop low')

fplt.plot(df['time'], volatility[1], ax=ax2, legend='volatility')
fplt.plot(df['time'], volatility_diff[1], ax=ax3, legend='volatility_diff')
# draw second plot
#for i, window in enumerate(windows): 
#    if(i<5):
#        fplt.plot(df['time'], volatility[i], ax=ax2, legend='volatility ' + str(window))
#    if(i>=5):
#        fplt.plot(df['time'], volatility[i], ax=ax3, legend='volatility ' + str(window))
#fplt.set_y_range(0, 0.005, ax=ax2) # hard-code y-axis range limitation

# draw third plot
#fplt.plot(df['time'], volatility_diff, ax=ax3, color='#927', legend='volatility_diff')

# draw fourth plot
#fplt.plot(df['time'], open_price_position_relative_to_volatility_box, ax=ax4, color='#927', legend='open_price_position_relative_to_volatility_box')

# restore view (X-position and zoom) if we ever run this example again
fplt.autoviewrestore()

# we're done
fplt.show()

## Create an API on Binance
From your online Binance profile -> API Management, create an API and give it some label. An API key and secret key
will be generated, there you can config some restrictions like a hardcode on the IP address to access and 'read' or
'withdrawal' permissions.

Once the API is created I save it in a JSON and don't share it with anyone, it´s better if you save it because the
secret key will be visible for sometime and then forever hidden, to dump in a json you can use the following simple
code:

In [ ]:
# create a json file with my keys
api_key = 'my_beautiful_key'
secret_key = 'life_is_wonderful'
filename = 'binance-python-credentials.json'

json.dump(dict(api_key=api_key, secret_key=secret_key), open(filename, 'w'))


# Profit simulation

In [3]:
import numpy as np
import random

num_trades = 10
success_ratio = 70/100

min_profit_perct = 2/100
max_loss_perct = 1/100

trades = np.array([])

# Wins
for x in range(int(num_trades * success_ratio)):
    r = (random.random() - 0.5)/100
    trade_profit_perct = min_profit_perct + r
    trades = np.append(trades, trade_profit_perct)

# Losses
for x in range(int(num_trades * (1 - success_ratio))):
    r = (random.random())/100
    trade_lost_perct = -(max_loss_perct + r - r)
    trades = np.append(trades, trade_lost_perct)

capital = 5e6
riesgo = 10/100

capital_arriesgado = capital * riesgo
print('Trade risk: ' + str(capital_arriesgado))

# Trade result
for x in range(num_trades):
    profit = trades[x] * capital_arriesgado
    trades[x] = profit

total_profit = np.sum(trades)
print('Daily profit: ' + str(total_profit))
print('Monthly profit: ' + str(total_profit*30))


Trade risk: 500000.0
Daily profit: 53514.236737111016
Monthly profit: 1605427.1021133305


# MA 200 + BB 200 strategy

* Plot MA 200 and BB 200 with (3 * standard deviation)
* When price breaks MA 200 enter two positions
    * Position 1 exits with a profit of 2%
    * Position 2 exits in next MA 200 cross

In [4]:
# Calculate MA 200
ma_200 = df['close'].rolling(200).mean()

# Calculate Bollinger Bands 200
bb_top_3delta = ma_200 + 3 * df['close'].rolling(200).std()
bb_bot_3delta = ma_200 - 3 * df['close'].rolling(200).std()

# Backtesting
pos = df['close'] > ma_200
pos_2  = df['position'].shift(1)
pos.dropna(inplace=True) # dropping the NaN values
pos_2.dropna(inplace=True)
df['crossover'] = np.where(pos == pos_2, False, True)

position_index = np.where(df['position']==True)
df['crossover'].iloc[position_index] = df['close'].iloc[position_index]

print(df['crossover'])

0         False
1         False
2         False
3         False
4         False
         ...   
4995    10147.7
4996    10144.6
4997    10148.8
4998    10148.2
4999      10154
Name: crossover, Length: 5000, dtype: object


In [5]:
try:
    # create two plots
    ax, ax2 = fplt.create_plot(symbol, rows=2)

    # plot candle sticks
    candles = df[['time','open','close','high','low']]
    fplt.candlestick_ochl(candles, ax=ax)

    # overlay volume on the top plot
    volumes = df[['time','open','close','volume']]
    fplt.volume_ocv(volumes, ax=ax.overlay())

    # Plot indicators
    fplt.plot(df['time'], ma_200, ax=ax, legend='ma_200')
    fplt.plot(df['time'], df['position'], ax=ax2, legend='crossover')
    #fplt.plot(df['time'], bb_bot_3delta, ax=ax, legend='bb_3d_bot')

    #fplt.plot(df['time'], df['close'].iloc[position_index], ax=ax, legend='position', style='^')
    #fplt.plot(crossing_dates, crossing_price, ax=ax2, legend='cross')

    # restore view (X-position and zoom)
    fplt.autoviewrestore()

    # we're done
    fplt.show()
except:
    pass

TypeError: numpy boolean subtract, the `-` operator, is not supported, use the bitwise_xor, the `^` operator, or the logical_xor function instead.